# template から HTML と JS を生成する


In [15]:
import os
from pathlib import Path
import json
import gzip
from collections import defaultdict

In [16]:
# Move up to top directory

print(os.getcwd())

while not Path("LICENSE").exists():
    os.chdir("../")

print(os.getcwd())

/mnt/e/Research/TSUMUGI-dev
/mnt/e/Research/TSUMUGI-dev


# 古いファイルの消去


In [17]:
%%bash

rm -rf TSUMUGI/data
rm -rf TSUMUGI/app

mkdir -p TSUMUGI/data/phenotype
mkdir -p TSUMUGI/data/genesymbol
mkdir -p TSUMUGI/app/phenotype
mkdir -p TSUMUGI/app/genesymbol
mkdir -p TSUMUGI/app/genelist/

cp -r TSUMUGI/template/css TSUMUGI/app/ # copy app's CSS
cp -r TSUMUGI/template/js TSUMUGI/app/ # copy app's JS

# 10 sec

## index.html の描画に必要なファイルを生成する


In [18]:
mp_terms = {}
for path_mp_term in Path("data", "mp_term_name").glob("*.csv"):
    mp_term = path_mp_term.stem
    if not Path("data", "network", "mp_term_name", f"{mp_term}.json.gz").exists():
        continue
    mp_term_name_space = mp_term.replace("_", " ")
    mp_terms[mp_term_name_space] = mp_term

In [19]:
print(list(mp_terms)[:3])
print(len(mp_terms))  # 515

['abnormal abdominal wall morphology', 'abnormal adrenal gland morphology', 'abnormal allantois morphology']
517


In [20]:
json.dump(mp_terms, open("data/overlap/available_mp_terms.json", "w"), indent=2)

In [21]:
%%bash

cp data/overlap/available_mp_terms.json TSUMUGI/data/available_mp_terms.json
cp data/overlap/available_gene_symbols.txt TSUMUGI/data/available_gene_symbols.txt
cp data/annotation/symbol_mgiid.json TSUMUGI/data/marker_symbol_accession_id.json


## index.html を生成


In [22]:
%%bash

grep -v "REMOVE_THIS_LINE" TSUMUGI/template/template_index.html > TSUMUGI/index.html

# JSON のコピー


In [ ]:
%%bash

cp -r data/network/mp_term_name/* TSUMUGI/data/phenotype
cp -r data/network/gene_symbol/* TSUMUGI/data/genesymbol

# 1m30s

In [ ]:
%%bash

ls TSUMUGI/data/phenotype | wc -l
ls TSUMUGI/data/genesymbol | wc -l
# TSUMUGI v0.1.3 (IMPC Release 22.0)
# 517
# 4139

517
4139


# Template から HTML と JS を生成


In [ ]:
path_files = Path("TSUMUGI", "data", "phenotype").glob("*.json.gz")
binary_phenotypes = []
for path_file in path_files:
    mp_term_name = path_file.name.replace(".json.gz", " ")
    with gzip.open(path_file, "rt") as f:
        json_data: list[dict] = json.load(f)

    count_node_color = defaultdict(int)
    for data in json_data:
        if "node_color" not in data["data"]:
            continue
        count_node_color[data["data"]["node_color"]] += 1
    if len(count_node_color) == 1 and 1 in count_node_color:
        binary_phenotypes.append(mp_term_name)

print(len(binary_phenotypes))
print(binary_phenotypes[:3])
Path("data", "annotation", "binary_phenotypes.txt").write_text("\n".join(binary_phenotypes))

154
['abnormal_abdominal_wall_morphology ', 'abnormal_adrenal_gland_morphology ', 'abnormal_allantois_morphology ']


3761

In [ ]:
%%bash

########################################
# Phenotype
########################################


find data/mp_term_name -type f |
    sed "s|data/mp_term_name/||" |
    sed "s|.csv$||" |
    while read -r mp_term_name_underscore; do
        mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
        impc_url=$(awk -F "\t" -v mp="$mp_term_name_space" '$1 == mp {print $2}' data/annotation/mptermname_phenotypeurl.tsv)
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')
        # --------------------------
        # HTML
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのフィルターを削除
        if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
            cat TSUMUGI/template/phenotype/template_network_phenotype.html |
                sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_impc_url|${escaped_url}|g" |
                sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/app/phenotype/"$mp_term_name_underscore".html
        else
            cat TSUMUGI/template/phenotype/template_network_phenotype.html |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_impc_url|${escaped_url}|g" |
                sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/app/phenotype/"$mp_term_name_underscore".html
        fi

        # --------------------------
        # Javascript
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのスライダーを削除
        if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
            cat TSUMUGI/template/phenotype/template_network_phenotype.js |
                sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/app/phenotype/"$mp_term_name_underscore".js
        else
            cat TSUMUGI/template/phenotype/template_network_phenotype.js |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > TSUMUGI/app/phenotype/"$mp_term_name_underscore".js
        fi

    done
# 24.4s

In [ ]:
%%bash

########################################
# Gene Symbol
########################################

cat data/overlap/available_gene_symbols.txt |
    while read gene_symbol; do
        mgi_id=$(awk -F "\t" -v key="$gene_symbol" '$1 == key {print $2}' data/annotation/symbol_mgiid.tsv)
        impc_url="https://www.mousephenotype.org/data/genes/"${mgi_id}
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

        # HTML
        cat TSUMUGI/template/genesymbol/template_network_genesymbol.html |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_impc_url|${escaped_url}|g" |
            sed "s|XXX_genesymbol|${gene_symbol}|g" > TSUMUGI/app/genesymbol/"$gene_symbol".html

        # Javascript
        cat TSUMUGI/template/genesymbol/template_network_genesymbol.js |
            sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_genesymbol|${gene_symbol}|g" > TSUMUGI/app/genesymbol/"$gene_symbol".js
    done

# 5 min

In [ ]:
%%bash

########################################
# Gene List
########################################

# HTML
cat TSUMUGI/template/genelist/template_network_genelist.html |
    sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
    grep -v "REMOVE_THIS_LINE" > TSUMUGI/app/genelist/network_genelist.html

# Javascript
cat TSUMUGI/template/genelist/template_network_genelist.js |
    sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
    grep -v "REMOVE_THIS_LINE" > TSUMUGI/app/genelist/network_genelist.js


bash: line 9: TSUMUGI/app/genelist/network_genelist.html: No such file or directory
bash: line 14: TSUMUGI/app/genelist/network_genelist.js: No such file or directory


CalledProcessError: Command 'b'\n########################################\n# Gene List\n########################################\n\n# HTML\ncat TSUMUGI/template/genelist/template_network_genelist.html |\n    sed \'/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d\' |\n    grep -v "REMOVE_THIS_LINE" > TSUMUGI/app/genelist/network_genelist.html\n\n# Javascript\ncat TSUMUGI/template/genelist/template_network_genelist.js |\n    sed \'/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d\' |\n    grep -v "REMOVE_THIS_LINE" > TSUMUGI/app/genelist/network_genelist.js\n'' returned non-zero exit status 1.

## テスト用のデータを test-tsumugi にコピー


In [ ]:
%%bash

rm -rf test-tsumugi
mkdir -p test-tsumugi/

mkdir -p test-tsumugi/data/phenotype
mkdir -p test-tsumugi/data/genesymbol
mkdir -p test-tsumugi/app/phenotype
mkdir -p test-tsumugi/app/genesymbol

In [ ]:
%%bash

cp -r TSUMUGI/index.html test-tsumugi/
cp -r TSUMUGI/css test-tsumugi/
cp -r TSUMUGI/js test-tsumugi/
cp -r TSUMUGI/image test-tsumugi/
cp -r TSUMUGI/template/css test-tsumugi/app/
cp -r TSUMUGI/template/js test-tsumugi/app/

cp -r TSUMUGI/data/*.txt test-tsumugi/data/
cp -r TSUMUGI/data/*.json test-tsumugi/data/

In [ ]:
%%bash

########################################
# Phenotype
########################################

find data/mp_term_name -type f |
    sed "s|data/mp_term_name/||" |
    sed "s|.csv$||" |
    grep -e ^male_infertility -e ^increased_fasting -e "^prewe" | # <- ここで興味のあるphenotypeを選択
    while read -r mp_term_name_underscore; do
        echo $mp_term_name_underscore

        mp_term_name_space=$(echo "$mp_term_name_underscore" | sed "s|_| |g")
        impc_url=$(awk -F "\t" -v mp="$mp_term_name_space" '$1 == mp {print $2}' data/annotation/mptermname_phenotypeurl.tsv)
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

        # --------------------------
        # HTML
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのフィルターを削除
        if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
            cat TSUMUGI/template/phenotype/template_network_phenotype.html |
                sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_impc_url|${escaped_url}|g" |
                sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".html
        else
            cat TSUMUGI/template/phenotype/template_network_phenotype.html |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_impc_url|${escaped_url}|g" |
            sed "s|XXX_mp_term_name_space|${mp_term_name_space}|g" |
            sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".html
        fi

        # --------------------------
        # Javascript
        # --------------------------
        # すべての効果量が１の場合、表現型の重症度でのスライダーを削除
        if grep -q "^$mp_term_name_underscore" data/annotation/binary_phenotypes.txt; then
            cat TSUMUGI/template/phenotype/template_network_phenotype.js |
                sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".js
        else
            cat TSUMUGI/template/phenotype/template_network_phenotype.js |
                grep -v "REMOVE_THIS_LINE" |
                sed "s|XXX_mp_term_name_underscore|${mp_term_name_underscore}|g" > test-tsumugi/app/phenotype/"$mp_term_name_underscore".js
        fi

        # --------------------------
        # Data
        # --------------------------

        cp TSUMUGI/data/phenotype/"$mp_term_name_underscore".json.gz test-tsumugi/data/phenotype/
    done

increased_fasting_circulating_glucose_level
male_infertility
preweaning_lethality,_complete_penetrance
preweaning_lethality,_incomplete_penetrance


In [ ]:
%%bash

########################################
# Gene Symbol
########################################

cat data/overlap/available_gene_symbols.txt |
    grep -e Rab10 -e Asxl1 -e Ddx46 | # <- ここで興味のあるgene symbolを選択
    while read gene_symbol; do
        echo $gene_symbol

        mgi_id=$(awk -F "\t" -v key="$gene_symbol" '$1 == key {print $2}' data/annotation/symbol_mgiid.tsv)
        impc_url="https://www.mousephenotype.org/data/genes/"${mgi_id}
        escaped_url=$(echo "$impc_url" | sed 's/\//\\\//g')

        # HTML
        cat TSUMUGI/template/genesymbol/template_network_genesymbol.html |
        grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_impc_url|${escaped_url}|g" |
            sed "s|XXX_genesymbol|${gene_symbol}|g" > test-tsumugi/app/genesymbol/"$gene_symbol".html

        # Javascript
        cat TSUMUGI/template/genesymbol/template_network_genesymbol.js |
            sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
            grep -v "REMOVE_THIS_LINE" |
            sed "s|XXX_genesymbol|${gene_symbol}|g" > test-tsumugi/app/genesymbol/"$gene_symbol".js

        # Data
        cp TSUMUGI/data/genesymbol/"$gene_symbol".json.gz test-tsumugi/data/genesymbol/

    done

Asxl1
Ddx46
Rab10


In [ ]:
%%bash

########################################
# Gene List
########################################

# HTML
cat TSUMUGI/template/genelist/template_network_genelist.html |
    sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
    grep -v "REMOVE_THIS_LINE" > test-tsumugi/app/genelist/network_genelist.html

# Javascript
cat TSUMUGI/template/genelist/template_network_genelist.js |
    sed '/REMOVE_FROM_THIS_LINE/,/REMOVE_TO_THIS_LINE/d' |
    grep -v "REMOVE_THIS_LINE" > test-tsumugi/app/genelist/network_genelist.js


In [ ]:
%%bash

date +"%Y/%m/%d %H:%M:%S" # Last update

2025/03/19 12:50:18
